# Tensorflow学习之卷积神经网络

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 获取数据
mnist=input_data.read_data_sets('Data\day_6',one_hot=True)

Extracting Data\day_6\train-images-idx3-ubyte.gz
Extracting Data\day_6\train-labels-idx1-ubyte.gz
Extracting Data\day_6\t10k-images-idx3-ubyte.gz
Extracting Data\day_6\t10k-labels-idx1-ubyte.gz


In [3]:
# 定义一个初始化权重的函数
def weght_variables(shape):
    w=tf.Variable(tf.random_normal(shape=shape,mean=0.0,stddev=1.0))
    return w

In [4]:
# 定义一个初始化偏置的函数
def bias_variables(shape):
    b=tf.Variable(tf.constant(0.0,shape=shape))
    return b

In [5]:
# 定义卷积模型
# 1、准备数据的占位符
with tf.variable_scope('data'):
    x=tf.placeholder(tf.float32,[None,784])

    y_true=tf.placeholder(tf.int32,[None,10])

In [6]:
# 2、第一层卷积 5*5*1，32个，步长strides=1 激活、池化
with tf.variable_scope('conv1'):

    # 随机初始化权重,偏置32
    w_conv1=w_conv1=weght_variables([5,5,1,32])

    b_conv1=bias_variables([32])

    # 对x进行形状的改变[None,784] [None,28,28,1],改变形状时，不知道填-1
    x_reshape=tf.reshape(x,[-1,28,28,1])

    # 激活 [None,28,28,1]----->[None,28,28,32]
    x_relu1=tf.nn.relu(tf.nn.conv2d(x_reshape,w_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)

    # 池化 2*2 步长strides=2 [None,28,28,32]----->[None,14 14 32]
    x_pool1=tf.nn.max_pool(x_relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
# 3、第二层卷积 5*5*32 64个filter，strides=1,激活：tf.nn.relu 池化
with tf.variable_scope('conv2'):

    # 随机初始化权重,偏置64个
    w_conv2=w_conv1=weght_variables([5,5,32,64])

    b_conv2=bias_variables([64])

    # 卷积、激活、池化
    # [None,14,14,32]---->[None,14,14,64]
    x_relu2=tf.nn.relu(tf.nn.conv2d(x_pool1,w_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)

    # 池化 2*2，步长=2，[None,14,14,64]---->[None,7,7,64]
    x_pool2=tf.nn.max_pool(x_relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [8]:
# 4、定义全连接层 [None,7*7*64]*[7*7*64,10]+[10]=[None,10]
with tf.variable_scope('fc'):

    # 随机初始化权重和偏置
    w_fc=weght_variables([7*7*64,10])

    b_fc=bias_variables([10])

    # 修改形状 [None,7,7,64]---->[None,7*7*64]
    x_fc_reshape=tf.reshape(x_pool2,[-1,7*7*64])

    # 进行矩阵运算得出每个样本的10个结果
    y_predict=tf.matmul(x_fc_reshape,w_fc)+b_fc

In [9]:
# 定义模型，得到输出
def conv_fc():

   # 进行交叉熵损失计算
   # 求出所有样本损失，然后求平均值
    with tf.variable_scope('soft_cross'):
       # 求平均交叉熵损失
       loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_predict))

    # 梯度下降求损失
    with tf.variable_scope('optimizer'):
        train_op=tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

    # 计算精确度
    with tf.variable_scope('acc'):
        equal_list=tf.equal(tf.argmax(y_true,1),tf.argmax(y_predict,1))
        accuracy=tf.reduce_mean(tf.cast(equal_list,tf.float32))

    # 定义初始化变量op
    init_op=tf.global_variables_initializer()

    # 开启会话运行
    with tf.Session() as sess:
        sess.run(init_op)
        for i in range(1000):
            mnist_x,mnist_y=mnist.train.next_batch(50)
            # 运行train_op
            sess.run(train_op,feed_dict={x:mnist_x,y_true:mnist_y})
            print('训练%d次，准确率为%f：' % (i,sess.run(accuracy,feed_dict={x:mnist_x,y_true:mnist_y})))
    return None

In [10]:
conv_fc()

训练0次，准确率为0.120000：
训练1次，准确率为0.100000：
训练2次，准确率为0.060000：
训练3次，准确率为0.100000：
训练4次，准确率为0.120000：
训练5次，准确率为0.100000：
训练6次，准确率为0.040000：
训练7次，准确率为0.100000：
训练8次，准确率为0.160000：
训练9次，准确率为0.060000：
训练10次，准确率为0.060000：
训练11次，准确率为0.120000：
训练12次，准确率为0.020000：
训练13次，准确率为0.120000：
训练14次，准确率为0.080000：
训练15次，准确率为0.100000：
训练16次，准确率为0.080000：
训练17次，准确率为0.160000：
训练18次，准确率为0.100000：
训练19次，准确率为0.100000：
训练20次，准确率为0.140000：
训练21次，准确率为0.180000：
训练22次，准确率为0.140000：
训练23次，准确率为0.080000：
训练24次，准确率为0.160000：
训练25次，准确率为0.200000：
训练26次，准确率为0.080000：
训练27次，准确率为0.140000：
训练28次，准确率为0.200000：
训练29次，准确率为0.140000：
训练30次，准确率为0.160000：
训练31次，准确率为0.200000：
训练32次，准确率为0.180000：
训练33次，准确率为0.120000：
训练34次，准确率为0.160000：
训练35次，准确率为0.220000：
训练36次，准确率为0.060000：
训练37次，准确率为0.180000：
训练38次，准确率为0.200000：
训练39次，准确率为0.100000：
训练40次，准确率为0.160000：
训练41次，准确率为0.100000：
训练42次，准确率为0.180000：
训练43次，准确率为0.200000：
训练44次，准确率为0.220000：
训练45次，准确率为0.100000：
训练46次，准确率为0.120000：
训练47次，准确率为0.060000：
训练48次，准确率为0.140000：
训练49次，准确率为0.180000：
训练50次，准确率为